# Report 1
## Taylor Sheneman

In [1]:
from __future__ import print_function, division

% matplotlib inline

from thinkbayes2 import Hist, Pmf, Suite

## Problems from Probably Overthinking It

### Cookie Problem

In [2]:
cookies = Pmf(['Bowl 1', 'Bowl 2'])
cookies.Print()

Bowl 1 0.5
Bowl 2 0.5


In [3]:
cookies['Bowl 1'] *= .75
cookies['Bowl 2'] *= .5
cookies.Normalize()
cookies.Print()

Bowl 1 0.6
Bowl 2 0.4


### M&M Problem

In [4]:
bag1994 = dict(brown=30, yellow=20, red=20,
             green=10, orange=10, tan=10, blue=0)

bag1996 = dict(blue=24, green=20, orange=16,
               yellow=14, red=13, brown=13, tan=0)


suite = Suite(['Bag 1','Bag 2'])
suite.Print()

Bag 1 0.5
Bag 2 0.5


In [5]:
suite['Bag 1'] *= bag1994['yellow']
suite['Bag 2'] *= bag1996['yellow']
suite.Normalize()
suite['Bag 1'] *= bag1996['green']
suite['Bag 2'] *= bag1994['green']
suite.Normalize()
suite.Print()

Bag 1 0.740740740741
Bag 2 0.259259259259


### Elvis Problem

In [6]:
pmf = Pmf(dict(identical=0.08, fraternal=0.92))
pmf['identical'] *= 1
pmf['fraternal'] *= 0.5
pmf.Normalize()
pmf.Print()

fraternal 0.851851851852
identical 0.148148148148


### Crime Scene Blood

The priors here are indeterminate, I suppose it's the difference between the priors and posteriors that's the question

In [7]:
pmf = Pmf(dict(present = 0.1, absent=0.9))
pmf['present'] *= 1
pmf['absent'] *= 0.6
pmf.Normalize()
pmf.Print()

absent 0.84375
present 0.15625


Evidence yes, strong evidence no. Assuming I did that right.


### Smoker Problem

The missing information here is the priors: 14.8% of women in the US are smokers.

In [8]:
pmf = Pmf(dict(smoker=14.8, nonsmoker=85.2))
pmf['smoker'] *= 13
pmf['nonsmoker'] *= 1
pmf.Normalize()
pmf.Print()

nonsmoker 0.306916426513
smoker 0.693083573487


### Monty Hall

In [9]:
class MontyHall(Suite):

    def Likelihood(self, data, hypo):
        """ x is the probability that Monty chooses door B, given that
        door A (the original choice) is the correct one. In the
        classical problem this is uniformly random, at 50% door B"""
        x = 0.5
        if hypo == data:
            return 0
        elif hypo == 'A':
            return x
        else:
            return 1

In [10]:
pmf = MontyHall('ABC')
pmf.Update('B')
pmf.Print()

A 0.333333333333
B 0.0
C 0.666666666667


## Another Bayes Problem (that I made up)

### Geographical Terrorism

In 2013, 82% of the 17,958 deaths due to terrorist attacks occured in five countries: Iraq, Afghanistan, Pakistan, Nigeria and Syria. Iraq is the worst affected, with 35.4% of the total terrorism-related fatalities.

A person dies in Iraq. How likely is it that they've died to a terrorist attack?

The missing information here: proportion of terrorism deaths to total deaths, proportion of non-terrorism deaths that occur in Iraq.

As it turns out, the death rate in 2013 was roughly 7.9 deaths/1000 people, with ~7 billion total world population and 33.42 million people in Iraq. This works out to about 55,300,000 total deaths in 2013, and about 264,000 deaths in Iraq. I'm assuming that worldwide average rate can reasonably represent deaths from non-terrorism causes in Iraq.

In [15]:
pmf = Pmf(dict(t_death=0.0003,n_death=0.9997))
pmf['t_death'] *= 35.4
pmf['n_death'] *= 0.48
pmf.Normalize()
pmf.Print()

n_death 0.978347564407
t_death 0.0216524355932


This kind of worked out into a problem that could be solved fairly simply without Bayes, just by virtue of being able to figure out the total terrorism deaths and total deaths in Iraq alone. I feel like there's potential for an interesting problem here though, just maybe by reworking the question? Still interesting, and the percentages approximately work out to the proportion of (Iraqi terrorism deaths)/(total Iraqi deaths).